# LU Decomposition

As you have seen in class over the last week or so, LU decomposition of matrices is a great way to systematically solve systems of equations. Along the way, you discovered *matrix inverses*, *permutation matrices*, *identity matrices*, and *upper and lower triangular matrices*.

Today, we will implement an LU decomposition algorithm and use it to solve systems of equations.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

## Row Reduction and LU Decomposition

In class, you saw that the $U$ part of the LU decomposition of a matrix is just the row-reduced form. You already have code for this! Figuring out the $L$ part is a matter of encoding the steps of the row reduction in matrix form. (There is also pivoting, which makes up the $P$ matrix in $PA = LU$. We will get back to that later.)

Find your `rowred(A)` code and paste it into the code box below. You'll be doing this sort of thing quite a lot, so it's worth saving all your routines into a file. If you save into a file called *referencefunctions.py*, you can run commands like `from referencefunctions import rowred`.

In [3]:
import numpy as np
from Qiureferencefunctions import rowred, rowaddmult, swaprows,backsub,rowredpivot



**Question 1** 
1. Why does this code already give you the $U$ matrix?<br><br>
1. By looking back at your class notes, explain what you need to do in order to generate the $L$ matrix:
  * You can initialize $L$ to be either a matrix of zeros or the identity matrix. For later purposes, we will choose to initialize to zeros.<br><br>
  * What entries should be added to it? When should they be added? Where in the matrix? Write down how the $L$ matrix changes with each step of the row-reduction.<br><br>
  * Since we started with a matrix of zeros, there is one last step needed after the code is done running in order to complete the $L$ matrix. What is it?<br><br>

**Question 2** By hand, compute the LU decomposition (without pivoting) of the matrix <br><br>

$$\begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 10 \end{bmatrix}$$

#### Some Python Notes

* Recall that the `np.zeros((m,n))` command returns an $m \times n$ matrix full of zeros. To get a matrix full of zeros that has the same shape as a matrix *A*, you can use `np.zeros_like(A)`. This is more efficient than getting the shape of *A* first.<br><br>

* You can get a $10\times 10$ diagonal matrix with all diagonal entries being 8 as follows:
```python
A=np.zeros((10,10))
np.fill_diagonal(A,8.)
```
  Note the dot after the *8*. What does it do?<br><br>
* To return more than one output, you can return a *tuple* object like $(A,B)$. You can then run code like:
```python
L,U=LUnopivot(A)
```

  You previously saw this notation when using the command `A.shape`.

As always, you should read the documentation for the commands mentioned above before using them.

**Question 3** 
1. Copy your row reduction code into the code box below, and rename the function `LUnopivot(A)`.<br><br>
1. Make the necessary changes to your code to transform it into LU decomposition. All you really need to do is create and fill the $L$ matrix!<br><br>
1. Test your code in two ways:<br><br>
  * By running it on the matrix above and comparing your answer to what you got by hand computation.<br><br>
  * By multiplying your $L$ and $U$ matrices. What should you get? Do you?

In [4]:
A = np.arange(1,10).reshape((3,3))
def LUnopivot(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    zero = np.zeros((rows,cols))
    cool = (copy,zero)
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            multval = (-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])
            rowaddmult(copy,pivotrow,i,(multval))
            zero[i,pivotrow] = -multval
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    np.fill_diagonal(zero,1.)
    return cool
print(LUnopivot(A))


(array([[ 1,  2,  3],
       [ 0, -3, -6],
       [ 0,  0,  0]]), array([[1., 0., 0.],
       [4., 1., 0.],
       [7., 2., 1.]]))


### The Need for Pivoting

Consider the following matrix from the last homework: $$\begin{bmatrix} 10^{-4} & 0 & 10^4 \\ 10^4 & 10^{-4} & 0 \\ 0 & 10^4 & 1\end{bmatrix}$$
In that lab, you saw that the row-reduction code you wrote makes a mess of a system of equations involving this matrix. That led us to implement pivoting.

**Question 4** Compute the LU decomposition of this matrix by hand.

**Question 5**  
1. Run your `LUnopivot(A)` function on this matrix.<br><br>
1. Examine $L$ and $U$. Are they the same as your hand computation above?<br><br>
1. Output $LU$. Do you get the original matrix back?

Those floating point errors are a pretty serious problem!

In [5]:
a = np.array([[0.0001,0,10000],[10000,.0001,0],[0,10000,1]])
both = LUnopivot(a)
print(both[0])
print(both[1])
print(both[0]@both[1])

[[ 1.e-04  0.e+00  1.e+04]
 [ 0.e+00  1.e-04 -1.e+12]
 [ 0.e+00  0.e+00  1.e+20]]
[[1.e+00 0.e+00 0.e+00]
 [1.e+08 1.e+00 0.e+00]
 [0.e+00 1.e+08 1.e+00]]
[[ 1.e-04  1.e+12  1.e+04]
 [ 1.e+04 -1.e+20 -1.e+12]
 [ 0.e+00  1.e+28  1.e+20]]


### Adding Pivoting to the Algorithm

The above algorithm takes a matrix $A$ and returns a decomposition $A=LU$. Recall that *pivoting* means that before we use a given row to reduce rows below it, we first swap the row with the largest absolute value in the column with the row we are currently at. To keep track of row swaps, we use a *permutation matrix*. 

**Question 6** Suppose that a $4\times 4$ matrix $B$ is derived from a matrix $A$ by first swapping rows 1 and 3, then swapping rows 2 and 3, then swapping rows 3 and 4. 

1. Write down a permutation matrix $P$ and a matrix equation expressing the relationship between $A$, $B$, and $P$.<br><br>

1. Explain how you arrived at your matrix for $P$.

**Question 7** Consider the first matrix from the last homework: $$\begin{bmatrix} 1 & 2 & 3 & -2\\ 2 & 4 & 1 &  0\\ 3 & 3 & 2 & 5 \\ -1 & 6 & 2 & 1\end{bmatrix}.$$ 

1. Run `LUnopivot(A)` on it. Why does it fail?<br><br>
1. Use MPP to compute the *PA=LU* decomposition of this matrix by hand. You can refer to Question 5 from the last homework if you like.

In [6]:
A = np.array([[1.,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]])

print (LUnopivot(A)) 


(array([[  1.,   2.,   3.,  -2.],
       [  0.,   0.,  -5.,   4.],
       [ nan,  nan, -inf,  inf],
       [ nan,  nan,  nan,  nan]]), array([[  1.,   0.,   0.,   0.],
       [  2.,   1.,   0.,   0.],
       [  3., -inf,   1.,   0.],
       [ -1.,  inf,  nan,   1.]]))


<ipython-input-4-58fb8c6dc28a>:12: RuntimeWarning: divide by zero encountered in double_scalars
  multval = (-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])
C:\Users\miche\2021-2022 School Year\Matrices and Vectors LAB\Qiureferencefunctions.py:7: RuntimeWarning: invalid value encountered in multiply
  A[j] = A[i]*c + A[j]
<ipython-input-4-58fb8c6dc28a>:12: RuntimeWarning: invalid value encountered in double_scalars
  multval = (-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])


Your `rowredpivot(A)` function does row reduction with pivoting. We will modify this function to compute LU decomposition with pivots. Start by pasting in the `rowredpivot(A)` function into the code box belowlike you did the `rowred(A)` function above. Rename the function `LU(A)`.

**Question 8** 
1. First, make the same modifications to the code that you made above. That is, add the initialization and filling of the $L$ matrix, as well as the changes needed to return both matrices.<br><br>
1. Next, we will need to add intializing the permutation matrix $P$. What should it be initially? Add in the code to initialize it. You may want to read the documentation for the `np.eye(n)` command...<br><br>
1. Next, we need to make sure we permute all the matrices. $U$ is already permuted due to our previous pivoting code. Add lines to permute the other two matrices.<br><br>
1. Lastly, we need to return all three matrices. Modify your code to do this.

In [10]:
A = np.array([[1.,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]])
def LU(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    zero = np.zeros((rows,cols))
    perm = np.eye(rows)
    cool = (copy,zero,perm) #U,L,P
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            maxe = np.argmax(abs(copy[pivotrow:,pivotcol])) +pivotrow
            if (maxe > pivotrow):
                swaprows(perm,maxe,pivotrow)
                swaprows(zero,maxe,pivotrow)
                copyrow = (copy[pivotrow]).copy();
                copy[pivotrow] = (copy[maxe]).copy();
                copy[maxe] = copyrow;
            multval = (-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])
            rowaddmult(copy,pivotrow,i,(multval))
            zero[i,pivotrow] = -multval
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    np.fill_diagonal(zero,1.)
    return cool;
x,y,z = LU(A)
print(x,y,z)
print()
print(A)
print (z@A)
print(y@x)

[[ 3.          3.          2.          5.        ]
 [ 0.          7.          2.66666667  2.66666667]
 [ 0.          0.          1.95238095 -4.04761905]
 [ 0.          0.          0.         -6.36585366]] [[ 1.          0.          0.          0.        ]
 [-0.33333333  1.          0.          0.        ]
 [ 0.33333333  0.14285714  1.          0.        ]
 [ 0.66666667  0.28571429 -0.56097561  1.        ]] [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

[[ 1.  2.  3. -2.]
 [ 2.  4.  1.  0.]
 [ 3.  3.  2.  5.]
 [-1.  6.  2.  1.]]
[[ 3.  3.  2.  5.]
 [-1.  6.  2.  1.]
 [ 1.  2.  3. -2.]
 [ 2.  4.  1.  0.]]
[[ 3.  3.  2.  5.]
 [-1.  6.  2.  1.]
 [ 1.  2.  3. -2.]
 [ 2.  4.  1.  0.]]


**Question 9** Test your code out by running it on the matrix given above:
1. Check that you get the same result as you got in your hand computation from Question 7.<br><br> 
1. Output both sides of the equation defining LU decomposition with pivoting and check they are equal.<br><br>
> **Important Note:** If you did everything correctly, you won't get exact equality: a number that is zero in the original is a very small decimal in *LU*. This has to do with another problem with floating point numbers: *binary representation*. We won't go into this in much detail, but sufice it to say that this decimal misrepresentation is common. Therefore, we do not check if two matrices are exactly equal. Instead, to check if two matrices are close to within a small tolerance, use `np.allclose(A,B)`.
<br><br>
1. Note that you do *not* get the original matrix back by multiplying *L* by *U*. What is the relationship between the matrix $LU$ and the original matrix?<br><br>
1. How can you use the matrix $P$ to recover the original matrix?

In [7]:
A = np.array([[1.,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]])

print(decomposition)
print(decomposition[2]@A,"\n", decomposition[1]@decomposition[0])
print(decomposition[2]@ decomposition[1]@decomposition[0])

NameError: name 'decomposition' is not defined

## Using LU Decomposition to Solve Equations

As you have seen in class, one of the most common uses of LU decomposition is in the solution of systems of linear equations. In this section, we will use the code you developed above to see how this works in practice. In the next section, we will apply this to fitting functions to data.

First, a reminder of how this works:

* Given a system of linear equations, convert it into matrix form $Ax=b$;<br><br>
* Decompose $A=LU$ (we'll get back to pivoting on the homework);<br><br>
* Then $Ax=b \Leftrightarrow LUx=b$;<br><br>
* Let $y=Ux$. Then $Ly=b$. Solve this equation by forward-substitution to find $y$.<br><br>
* Lastly, solve $Ux=y$ by back-substitution to find $x$.<br><br>

We have code for the first three steps and the last one. We'll need code for the fourth...

### Forward Substitution

**Question 10** Write a function `fwdsub(L,v)` that takes an lower-triangular matrix $L$ and a vector $v$ and returns the solution of $Lx=v$. This should be a relatively small modification of your `backsub(U,v)` function. Show a test of your code.

In [ ]:
A = np.array([[1.,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]])
v = np.array([4,6,7,8])
def fwdsub(L,v):
    rows,cols = L.shape
    x = np.zeros(cols)
    for i in range (0,rows,1):
        x[i] += (L[i, :i]@x[:i]) # dot product version 
        x[i] = v[i] - x[i]
        x[i] = x[i]/L[i,i]
    return x;
print(decomposition[1])
print(fwdsub(decomposition[1],v))


**Question 11** Use your LU decomposition code (use `LUnopivot(A)` this time) and your `backsub(U,v)` and `fwdsub(L,v)` functions to solve the following system of equations:<br><br>
$$\begin{array}
4x_1 + 6x_2 - x_3 + 2x_4 & = & -22\\
-x_1 + 9x_2 + 7x_3 - 6x_4 & = & -26\\
2x_1 + x_2 + 4x_3 - 2x_4 & = & -20\\
9x_1 + 6x_2 + 3x_3 - 7x_4 & = & -34\\
\end{array}$$

In [ ]:
a = np.array([[1.,6,-1,2],[-1,9,7,-6],[2,1,4,-2],[9,6,3,-7]])
v = np.array([-22,-26,-20,-34])
rows,cols = a.shape #dimensions of the original matrix yet to be augmented
#A = np.zeros(rows*(cols+1)).reshape(rows,cols+1)
#A[:rows,:cols] = a[:,:]
#A[:,cols] = v
sol = LUnopivot(a)
L = sol[1]
U = sol[0]
print(sol)
y = fwdsub(L,v)
soln = backsub(U,y)
print(soln)
print(a@soln)

### Fitting Polynomial Curves to Data

Suppose that we have $n$ data points from an experiment. Each data point consists of a pair of numbers. We will see that using solutions of linear systems of equations, we can fit an $(n-1)$ degree polynomial to the points.

#### Back to High School

**Question 12** Suppose you only have two data points: $(1,2)$ and $(4,10)$. 
1. What is $n$?<br><br>
1. In this case, what does an $(n-1)$ degree polynomial's graph look like?<br><br>
1. Find the $(n-1)$ degree polynomial that fits this data using knowledge from high school.

#### Using Linear Algebra

In general, an $n$ degree polynomial can be written: $$p(x)=a_nx^n + a_{n-1}x^{n-1} + \ldots a_1 x + a_0\mbox{, where } a_n\neq 0$$

**Question 13** Using the above two data points, we can see that a first degree polynomial that fits them must satisfy the system of equations (make sure you see why!): 
$$\begin{array}
~a_1\times1 + a_0 & = & 2 \\
a_1\times4 + a_0 & = & 10\\
\end{array}$$
Solve this system and check you get the same answer as above.

**Question 14** Suppose that in addition to the above two data points, we also had $(5,1)$. Write down and solve a system of equations to find a quadratic (second degree) polynomial that fits these data points.

**Question 15** Find the best polynomial to fit the following data. You are probably going to want to write the data as a system of linear equations and use your earlier code.

$x$ | -3 | -2 | -1 | 0 | 1 | 2 | 3
--- | :---: | :---: | :---: |:---: |:---: |:---: |:---: |
$y$ | -15.991 | -4.36 | -1.603 | -1 | -1.111 | -2.536 | -9.715

**Question 16** Suppose you had the following data points: $(1,2)$, $(2,4.1)$, and $(3,5.9)$. While you could use the ideas we just developed to fit a quadratic function to this data, explain why this is probably misguided. (Hint: you may want to plot these points!)

We will return to the idea of fitting curves to data in Lab 5, when we talk about *least squares*. 

In [8]:
a = np.array([-3,-2,-1, 0,1,2,3])
v = np.array([-15.991,-4.36,-1.603,-1,-1.111,-2.536, -9.715])
rows = a.shape[0]
AA = np.zeros((rows,rows))
for i in range(0,rows):
    x = pow(a,i)
    AA[:,i] = x
print(AA)
AA =np.fliplr(AA)
#A = np.zeros((rows,rows+1))
#A[:,:rows] = AA
#A[:,rows] = v
np.set_printoptions(suppress=True)

print(AA)
U,L,P = LU(AA)
print(U)
v = P@v
y = fwdsub(L,v)
print(backsub(U,y))

[[   1.   -3.    9.  -27.   81. -243.  729.]
 [   1.   -2.    4.   -8.   16.  -32.   64.]
 [   1.   -1.    1.   -1.    1.   -1.    1.]
 [   1.    0.    0.    0.    0.    0.    0.]
 [   1.    1.    1.    1.    1.    1.    1.]
 [   1.    2.    4.    8.   16.   32.   64.]
 [   1.    3.    9.   27.   81.  243.  729.]]
[[ 729. -243.   81.  -27.    9.   -3.    1.]
 [  64.  -32.   16.   -8.    4.   -2.    1.]
 [   1.   -1.    1.   -1.    1.   -1.    1.]
 [   0.    0.    0.    0.    0.    0.    1.]
 [   1.    1.    1.    1.    1.    1.    1.]
 [  64.   32.   16.    8.    4.    2.    1.]
 [ 729.  243.   81.   27.    9.    3.    1.]]
[[ 729.         -243.           81.          -27.            9.
    -3.            1.        ]
 [   0.          486.            0.           54.            0.
     6.            0.        ]
 [   0.            0.            8.88888889    4.44444444    3.20987654
     1.60493827    0.9122085 ]
 [   0.            0.            0.           -8.88888889    0.
    -3.2098

NameError: name 'fwdsub' is not defined